<a href="https://colab.research.google.com/github/thedarsideofit/finance_python_practices/blob/main/prophet_aapl_forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Main Source

https://towardsdatascience.com/apple-stock-and-bitcoin-price-predictions-using-fbs-prophet-for-beginners-python-96d5ec404b77?gi=962fdc51375b

In [ ]:
pip install alphavantage_api

In [ ]:
pip install alpha_vantage

In [ ]:
import pandas as pd
from fbprophet import Prophet
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import wordcloud
from alphaVantageAPI.alphavantage import AlphaVantage 

In [ ]:
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt
ts = TimeSeries(key='Your Access Key',output_format='pandas')
apple, meta_data = ts.get_daily(symbol='AAPL',outputsize='full')
apple.head()

In [ ]:
#Write/Read from a file.
apple.to_csv ('/content/drive/My Drive/Colab Notebooks/Finanzas/pricedataapple.csv', index = 'date', header=True)
apple = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Finanzas/pricedataapple.csv')
apple.head()

In [ ]:
from alpha_vantage.techindicators import TechIndicators
import matplotlib.pyplot as plt
ti = TechIndicators(key='NLRHFAQBGU5WGPQH',output_format='pandas')
data, meta_data = ti.get_sma(symbol='AAPL',interval='daily', time_period=60,series_type = 'close')
data.plot()
plt.show()

In [ ]:
from alpha_vantage.techindicators import TechIndicators
import matplotlib.pyplot as plt
ti = TechIndicators(key='Youraccesskey',output_format='pandas')
data, meta_data = ti.get_rsi(symbol='AAPL',interval='daily', time_period=60,series_type = 'close')
data.plot()
plt.show()

In [ ]:
#Convert the dataframe into the prophet expected format.
apple.rename(columns={'4. close': 'y', 'date': 'ds'},inplace=True)
apple['ds'] = pd.to_datetime(apple['ds'])
apple['y']=apple['y'].astype(float)

In [ ]:
#Fit the model - df-Apple
df_prophet = Prophet(changepoint_prior_scale=0.15, daily_seasonality=True)
df_prophet.fit(apple)

In [ ]:
fcast_time=20   # 1 year
df_forecast = df_prophet.make_future_dataframe(periods= fcast_time, freq='D')
df_forecast.tail(10)

In [ ]:
# Forecasting - call the method predict
df_forecast = df_prophet.predict(df_forecast)
df_forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(60)

In [ ]:
df_prophet.plot(df_forecast, xlabel = 'Date', ylabel = 'Apple Price')

In [ ]:
fig2 = df_prophet.plot_components(df_forecast)
plt.show()

In [ ]:
from fbprophet.plot import plot_plotly
import plotly.offline as py
py.init_notebook_mode()

fig = plot_plotly(df_prophet, df_forecast)  # This returns a plotly Figure
py.iplot(fig)

In [ ]:
from fbprophet.diagnostics import cross_validation
df_cv = cross_validation(df_prophet, initial='2920 days', period='60 days', horizon = '720 days')
df_cv.tail()

In [ ]:
from fbprophet.diagnostics import performance_metrics
df_p = performance_metrics(df_cv)
df_p.head()

In [ ]:
from fbprophet.plot import plot_cross_validation_metric
fig = plot_cross_validation_metric(df_cv, metric = 'mape')